In [73]:
import sys
import json
import requests
import time
import argparse
import pandas as pd
import numpy as np
import pymongo

import random

import boto3
import boto.s3.connection
from boto.s3.key import Key
from src.Airbnb_Scraping_tools import *
from urllib.parse import urlparse
from urllib.request import urlretrieve 

In [74]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load in data

In [75]:
s3_path = 'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [76]:
s3_path

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [77]:
folder = 'Boston-Massachusetts-US'
file_tmp = pd.read_csv(s3_path+folder+'/listings.csv.gz',compression = 'gzip')
print('Total number of apartments in ' + folder +" is: {}".format(len(file_tmp)))
file_tmp.describe()

Total number of apartments in Boston-Massachusetts-US is: 4870


,id,scrape_id,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,jurisdiction_names,calculated_host_listings_count,reviews_per_month
count,4.870000e+03,4.870000e+03,4.870000e+03,0.0,4870.000000,4870.000000,0.0,4870.000000,4870.000000,4870.000000,...,3936.000000,3941.000000,3928.000000,3940.000000,3927.000000,3927.000000,0.0,0.0,4870.000000,3986.000000
mean,1.293651e+07,2.017101e+13,4.088556e+07,NaN,78.667351,78.667351,NaN,42.339999,-71.083943,3.298152,...,9.588923,9.443035,9.742617,9.713706,9.487395,9.334861,NaN,NaN,19.301848,2.204792
std,6.202199e+06,5.000513e-01,3.971610e+07,NaN,219.284219,219.284219,NaN,0.025338,0.032768,2.131683,...,0.773592,0.957797,0.685026,0.722654,0.822990,0.890983,NaN,NaN,43.029043,2.336672
min,3.781000e+03,2.017101e+13,4.240000e+03,NaN,0.000000,0.000000,NaN,42.235760,-71.179257,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,NaN,NaN,1.000000,0.020000
25%,7.913572e+06,2.017101e+13,1.011587e+07,NaN,1.000000,1.000000,NaN,42.329070,-71.105007,2.000000,...,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,NaN,1.000000,0.450000
50%,1.431376e+07,2.017101e+13,2.695608e+07,NaN,2.000000,2.000000,NaN,42.345512,-71.077124,2.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,9.000000,NaN,NaN,2.000000,1.300000
75%,1.857908e+07,2.017101e+13,6.297457e+07,NaN,11.000000,11.000000,NaN,42.356009,-71.061114,4.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,9.000000,3.247500
max,2.123117e+07,2.017101e+13,1.533133e+08,NaN,982.000000,982.000000,NaN,42.396191,-70.985047,16.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,NaN,181.000000,15.100000


# Open S3 bucket

In [78]:
bucket_name = "chen-gal-test"

In [79]:
s3 = boto3.client("s3")

remote_pathname_l1 = 'AirbnbData/'
remote_pathname_l2 = folder
remote_pathname = remote_pathname_l1 + remote_pathname_l2

# s3.put_object(Body=file , Bucket=bucket_name, Key=remote_pathname)


In [80]:
logfile_name = 'DataSet/'+'logfile_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')




Logfile name is DataSet/logfile_Boston-Massachusetts-US.txt


4870

# Downloading

In [84]:

apt_infos = {}
apt_jses = {}

In [112]:
for i in range(256,len(file_tmp)):
# for i in range(1):
    print(i)
    url = file_tmp.loc[i,'listing_url']
    apt_id = file_tmp.loc[i,'id']
    apt_name = file_tmp.loc[i,'name']
    with open(logfile_name,'a') as f:
        f.write(f'---------------------\nApartment{i} | {apt_id} | {url} \n')         
    try:
        return_parsed = parse_each_webpage(url)
    except CannotOpenUrl:        
        with open(logfile_name,'a') as f:
            f.writelines('Cannot open '+ str(i)+ ' url.\t' + 'Id: '+ str(apt_id) + ' |  Url: '+ url)                        
        print('Failed to get access to URL, sleep for 20 secs ')
        time.sleep(20)
        return_parsed = parse_each_webpage(url)
    finally:
        scrape_info,ori_json = return_parsed[0],return_parsed[1]
        if len(scrape_info) == 0:
            with open(logfile_name,'a') as f:
                f.writelines('Failed to scrape the url, simply save the json file\n')
        else:
            with open(logfile_name,'a') as f:
                f.writelines('Successfully scraped the url, save both files\n')
            
        dict_apt = {'apt_name':apt_name,'url' : url, 'info': scrape_info}
        apt_jses.update({str(apt_id):ori_json})
        apt_infos.update({str(apt_id):dict_apt})
        with open(logfile_name,'a') as f:
                f.writelines(f'Finished scraping {i} apartment! ID:{apt_id}\n')
                
        if i % 75 == 0 and i != 0:            
            js_filename = remote_pathname +"/jsons/ori_json_"+str(i)            
            s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=js_filename)
            
            print("Wrote  down"+str(i)+"json file")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved json file {js_filename}!\n') 
        if i % 200 == 0 and i != 0:            
            wsdata_filename = remote_pathname +"/ws_data/scraped_partial_data_"+str(i)            
            s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=wsdata_filename)
            
            print("Wrote  down"+str(i)+"ws_data")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved ws_data file {js_filename}!\n')  
            
        time.sleep(random.random()+2)
        
#upload log_file to s3
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)


256
request_url:https://www.airbnb.com/rooms/6990904
257
request_url:https://www.airbnb.com/rooms/3730447
258
request_url:https://www.airbnb.com/rooms/568234
259
request_url:https://www.airbnb.com/rooms/9549850
260
request_url:https://www.airbnb.com/rooms/16362002
261
request_url:https://www.airbnb.com/rooms/9349646
262
request_url:https://www.airbnb.com/rooms/9349614
263
request_url:https://www.airbnb.com/rooms/13652374
264
request_url:https://www.airbnb.com/rooms/9590679
265
request_url:https://www.airbnb.com/rooms/13137300
266
request_url:https://www.airbnb.com/rooms/16494984
267
request_url:https://www.airbnb.com/rooms/20572354
268
request_url:https://www.airbnb.com/rooms/7956634
269
request_url:https://www.airbnb.com/rooms/20784807
270
request_url:https://www.airbnb.com/rooms/13363171
271
request_url:https://www.airbnb.com/rooms/4223387
272
request_url:https://www.airbnb.com/rooms/14868157
273
request_url:https://www.airbnb.com/rooms/10627514
274
request_url:https://www.airbnb.com

409
request_url:https://www.airbnb.com/rooms/19830344
410
request_url:https://www.airbnb.com/rooms/15338795
411
request_url:https://www.airbnb.com/rooms/16289538
412
request_url:https://www.airbnb.com/rooms/2592416
413
request_url:https://www.airbnb.com/rooms/20205227
414
request_url:https://www.airbnb.com/rooms/19910241
415
request_url:https://www.airbnb.com/rooms/12602632
416
request_url:https://www.airbnb.com/rooms/15126054
417
request_url:https://www.airbnb.com/rooms/5738018
418
request_url:https://www.airbnb.com/rooms/6065674
419
request_url:https://www.airbnb.com/rooms/12209652
420
request_url:https://www.airbnb.com/rooms/19568535
421
request_url:https://www.airbnb.com/rooms/14683493
422
request_url:https://www.airbnb.com/rooms/18812187
423
request_url:https://www.airbnb.com/rooms/5049582
424
request_url:https://www.airbnb.com/rooms/3591359
425
request_url:https://www.airbnb.com/rooms/12209339
426
request_url:https://www.airbnb.com/rooms/20073664
427
request_url:https://www.airbn

561
request_url:https://www.airbnb.com/rooms/11313921
562
request_url:https://www.airbnb.com/rooms/19980998
563
request_url:https://www.airbnb.com/rooms/21109844
564
request_url:https://www.airbnb.com/rooms/13525610
565
request_url:https://www.airbnb.com/rooms/19173476
566
request_url:https://www.airbnb.com/rooms/5526487
567
request_url:https://www.airbnb.com/rooms/19177794
568
request_url:https://www.airbnb.com/rooms/19561932
569
request_url:https://www.airbnb.com/rooms/18228183
570
request_url:https://www.airbnb.com/rooms/20173397
571
request_url:https://www.airbnb.com/rooms/14196983
572
request_url:https://www.airbnb.com/rooms/6611126
573
request_url:https://www.airbnb.com/rooms/19177986
574
request_url:https://www.airbnb.com/rooms/18599691
575
request_url:https://www.airbnb.com/rooms/5267396
576
request_url:https://www.airbnb.com/rooms/19456107
577
request_url:https://www.airbnb.com/rooms/5904992
578
request_url:https://www.airbnb.com/rooms/19455243
579
request_url:https://www.airb

713
request_url:https://www.airbnb.com/rooms/15187909
714
request_url:https://www.airbnb.com/rooms/11973196
715
request_url:https://www.airbnb.com/rooms/3342206
716
request_url:https://www.airbnb.com/rooms/13074357
717
request_url:https://www.airbnb.com/rooms/18546212
718
request_url:https://www.airbnb.com/rooms/15187617
719
request_url:https://www.airbnb.com/rooms/3341329
720
request_url:https://www.airbnb.com/rooms/12303347
721
request_url:https://www.airbnb.com/rooms/15283836
722
request_url:https://www.airbnb.com/rooms/11968079
723
request_url:https://www.airbnb.com/rooms/12284774
724
request_url:https://www.airbnb.com/rooms/17282016
725
request_url:https://www.airbnb.com/rooms/10116601
726
request_url:https://www.airbnb.com/rooms/19116365
727
request_url:https://www.airbnb.com/rooms/16806101
728
request_url:https://www.airbnb.com/rooms/18969917
729
request_url:https://www.airbnb.com/rooms/8924562
730
request_url:https://www.airbnb.com/rooms/17559493
731
request_url:https://www.air

865
request_url:https://www.airbnb.com/rooms/6089865
866
request_url:https://www.airbnb.com/rooms/16739513
867
request_url:https://www.airbnb.com/rooms/20032727
868
request_url:https://www.airbnb.com/rooms/17509246
869
request_url:https://www.airbnb.com/rooms/21228356
870
request_url:https://www.airbnb.com/rooms/12784064
871
request_url:https://www.airbnb.com/rooms/6134145
872
request_url:https://www.airbnb.com/rooms/14461498
873
request_url:https://www.airbnb.com/rooms/15444531
874
request_url:https://www.airbnb.com/rooms/9841140
875
request_url:https://www.airbnb.com/rooms/13248823
876
request_url:https://www.airbnb.com/rooms/3664031
877
request_url:https://www.airbnb.com/rooms/3434614
878
request_url:https://www.airbnb.com/rooms/20309868
879
request_url:https://www.airbnb.com/rooms/20932029
880
request_url:https://www.airbnb.com/rooms/18343565
881
request_url:https://www.airbnb.com/rooms/4089940
882
request_url:https://www.airbnb.com/rooms/20309505
883
request_url:https://www.airbnb

...

2212
request_url:https://www.airbnb.com/rooms/21024550
2213
request_url:https://www.airbnb.com/rooms/3781
2214
request_url:https://www.airbnb.com/rooms/17387160
2215
request_url:https://www.airbnb.com/rooms/19587539
2216
request_url:https://www.airbnb.com/rooms/13995462
2217
request_url:https://www.airbnb.com/rooms/16895403
2218
request_url:https://www.airbnb.com/rooms/21171599
2219
request_url:https://www.airbnb.com/rooms/15535385
2220
request_url:https://www.airbnb.com/rooms/12646288
2221
request_url:https://www.airbnb.com/rooms/18290558
2222
request_url:https://www.airbnb.com/rooms/4820071
2223
request_url:https://www.airbnb.com/rooms/20663729
2224
request_url:https://www.airbnb.com/rooms/21171163
2225
request_url:https://www.airbnb.com/rooms/9145242
2226
request_url:https://www.airbnb.com/rooms/20696002
2227
request_url:https://www.airbnb.com/rooms/20663083
2228
request_url:https://www.airbnb.com/rooms/3967690
2229
request_url:https://www.airbnb.com/rooms/16173765
2230
request_url:

2362
request_url:https://www.airbnb.com/rooms/17667949
2363
request_url:https://www.airbnb.com/rooms/6690605
2364
request_url:https://www.airbnb.com/rooms/5592866
2365
request_url:https://www.airbnb.com/rooms/20256823
2366
request_url:https://www.airbnb.com/rooms/17207702
2367
request_url:https://www.airbnb.com/rooms/6886946
2368
request_url:https://www.airbnb.com/rooms/14963976
2369
request_url:https://www.airbnb.com/rooms/2167993
2370
request_url:https://www.airbnb.com/rooms/14292385
2371
request_url:https://www.airbnb.com/rooms/18910691
2372
request_url:https://www.airbnb.com/rooms/8442940
2373
request_url:https://www.airbnb.com/rooms/6378076
2374
request_url:https://www.airbnb.com/rooms/21041723
2375
request_url:https://www.airbnb.com/rooms/19501578
2376
request_url:https://www.airbnb.com/rooms/1102623
2377
request_url:https://www.airbnb.com/rooms/1658983
2378
request_url:https://www.airbnb.com/rooms/15297238
2379
request_url:https://www.airbnb.com/rooms/18053113
2380
request_url:h

2511
request_url:https://www.airbnb.com/rooms/18387338
2512
request_url:https://www.airbnb.com/rooms/14200338
2513
request_url:https://www.airbnb.com/rooms/19019562
2514
request_url:https://www.airbnb.com/rooms/20410237
2515
request_url:https://www.airbnb.com/rooms/20639808
2516
request_url:https://www.airbnb.com/rooms/21172765
2517
request_url:https://www.airbnb.com/rooms/7633883
2518
request_url:https://www.airbnb.com/rooms/5690640
2519
request_url:https://www.airbnb.com/rooms/13249829
2520
request_url:https://www.airbnb.com/rooms/8052494
2521
request_url:https://www.airbnb.com/rooms/12365159
2522
request_url:https://www.airbnb.com/rooms/13797236
2523
request_url:https://www.airbnb.com/rooms/13778731
2524
request_url:https://www.airbnb.com/rooms/13185237
2525
request_url:https://www.airbnb.com/rooms/14742722
2526
request_url:https://www.airbnb.com/rooms/18957205
2527
request_url:https://www.airbnb.com/rooms/13121821
2528
request_url:https://www.airbnb.com/rooms/2754975
2529
request_u

2660
request_url:https://www.airbnb.com/rooms/16276224
2661
request_url:https://www.airbnb.com/rooms/14572100
2662
request_url:https://www.airbnb.com/rooms/18487889
2663
request_url:https://www.airbnb.com/rooms/10164810
2664
request_url:https://www.airbnb.com/rooms/11016623
2665
request_url:https://www.airbnb.com/rooms/15407458
2666
request_url:https://www.airbnb.com/rooms/21141875
2667
request_url:https://www.airbnb.com/rooms/21076367
2668
request_url:https://www.airbnb.com/rooms/16436951
2669
request_url:https://www.airbnb.com/rooms/15767548
2670
request_url:https://www.airbnb.com/rooms/8428052
2671
request_url:https://www.airbnb.com/rooms/16071075
2672
request_url:https://www.airbnb.com/rooms/5445126
2673
request_url:https://www.airbnb.com/rooms/5612512
2674
request_url:https://www.airbnb.com/rooms/19945100
2675
request_url:https://www.airbnb.com/rooms/19307193
2676
request_url:https://www.airbnb.com/rooms/11343485
2677
request_url:https://www.airbnb.com/rooms/9596153
2678
request_u

KeyboardInterrupt: 

In [113]:
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

In [102]:
with open(logfile_name,'a') as f:
    f.writelines(f'Saved json file {js_filename}!\n')                            
jses = {}

## Save the result

### save scrapted data

In [114]:
scraped_filename = remote_pathname +"/ws_data/webscrapted.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

{'ETag': '"3aa75aea13d6b1d083cea56b1332defc"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 14:25:10 GMT',
   'etag': '"3aa75aea13d6b1d083cea56b1332defc"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'nqSPf62ROozo7o1D8QySuFr+mmuIhuzPg3/kiYujauBn+ielf3j6ERSblVt947VSyobzVMm5EmA=',
   'x-amz-request-id': '0E52A610C8561BD5'},
  'HTTPStatusCode': 200,
  'HostId': 'nqSPf62ROozo7o1D8QySuFr+mmuIhuzPg3/kiYujauBn+ielf3j6ERSblVt947VSyobzVMm5EmA=',
  'RequestId': '0E52A610C8561BD5',
  'RetryAttempts': 0}}

### save json data

In [88]:
last_piece_js_filename = remote_pathname+"/jsons/ori_json_lastpiece.json"

s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=last_piece_js_filename)


{'ETag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 01:22:39 GMT',
   'etag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
   'x-amz-request-id': 'BB474E35A45B7DAC'},
  'HTTPStatusCode': 200,
  'HostId': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
  'RequestId': 'BB474E35A45B7DAC',
  'RetryAttempts': 0}}

## Load json data from s3

In [106]:
response = s3.get_object(Bucket=bucket_name,
                         Key=scraped_filename)

In [107]:
scraped_data = json.loads(response['Body'].read())

In [109]:
apt_infos = scraped_data